# HW7 과제 : 2021741022 신정석호
목표
텍스트 데이터를 표현하는 여러 가지 방법
- (Document-Term Matrix, TF-IDF Matrix, Word2Vec, Pre-Trained GloVe-Twitter-25)을 사용하여 
- 20news_group 데이터셋을 분류하고, 각 방법의 성능을 비교 및 분석한다

과제 설명
- 1.데이터 : fetch_20newsgroups 데이터셋을 찾아 로드한다.
- 2.Text Data 표현 방법을 다음의 4 가지를 이용하여 각각 분류 성능을 구하고 비교한다. 
- (분류 모델은 선형 및 대표적인 비선형 모델 몇 가지를 선택하고, 성능은 Accuray, F-1 Score, ROC/AUC 사용한다.)
- Document-Term Matrix (DTM):
- TF-IDF Matrix:
- Word2Vec() 을 이용하여 직접 임베딩 (시간이 많이 걸릴 수 있음)
- Pre-Trained GloVe-Twitter-25 사용 
- **데이터 로드 및 Text 변환 등은 제공해 준 소스 코드 (Topic Modeling 예제 소스)를 참고하세요.

# 0. 데이터 구조 파악

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_20newsgroups
# fetch_20newsgroups는 미국 뉴스그룹(뉴스 게시판) 글들로 구성된 텍스트 데이터셋을 제공합니다.
# 정확히는 Usenet이라는 인터넷 게시판에 올라온 글들을 20개의 주제로 분류한 데이터셋입니다.

In [2]:
data = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
# 20 Newsgroups는 train과 test 로 나뉘어 있다.
# headers(이메일, 제목, 발신자, 날짜같은 메타데이터( = 본문을 설명하는 정보 등) ) 
# footers ( 글 마지막에 붙는 서명 ) 
# quotes ( 이전 글을 인용한 부분 ) 

documents = data.data # 실제 텍스트 데이터
labels = data.target # 텍스트 분류 모델의 정답 라벨(y)


print(data.keys())
# data : 문서 본문 텍스트 리스트
# filenames : 각 문서가 원래 저장되어 있던 로컬 경로. 원본 데이터셋에서 해당 글이 어떤 파일에서 왔는지 알려줌 
# target_names : 20개의 카테고리 이름
# target : 각 문서의 라벨(숫자, 0-19). 텍스트 분류 모델의 정답 라벨(y). target_names의 index에 해당한다.
# DESCR : 데이터셋의 설명문

# 예시 : data.data[0]의 카테고리는 data.target[0]이다.
# data.target_names[data.target[0]]은 데이터의 카테고리 이름이다.


dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [3]:
# 데이터 구조 확인
print("데이터 구조 키:", data.keys())
print("총 문서 수:", len(data.data))
print("카테고리 수:", len(data.target_names))
print("카테고리 이름들:", data.target_names)


데이터 구조 키: dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])
총 문서 수: 11314
카테고리 수: 20
카테고리 이름들: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [4]:
for i in range(5):
    print(f'[{i}]  {data.data[i]}\n')

[0]  I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

[1]  A fair number of brave souls who upgraded their SI clock oscillator have
shared their experiences for this poll. Please send a brief message detailing
your experiences with the procedure. Top speed attained, CPU rated speed,
add on cards and adapters, heat sinks, hour of usage per day, floppy disk
functionality with 800 and 1.4 m floppies are especially requested.

I will be summarizing in the next two days, so please add to the network
knowledge base if you have done the clock upgrade and haven't answered t

In [5]:
print(labels)
print("Labels(y) shape:", labels.shape)
print("예시 라벨:", labels[:10])

[7 4 4 ... 3 1 8]
Labels(y) shape: (11314,)
예시 라벨: [ 7  4  4  1 14 16 13  3  2  4]


# 1. 텍스트 전처리
- 소문자 변환
- 불용어 제거
- 특수문자/숫자 제거
- 토근화 ( 문장을 단어 단위로 쪼개기)
- 어간 추출/ 표제어 추출 ( 단어 형태 통일. ex: running -> run)

In [6]:
!pip install nltk

In [7]:
from nltk.tokenize import word_tokenize  # 토큰화 함수
from nltk.corpus import stopwords # 불용어 제공
import nltk
import re # 정규 표현식 라이브러리 (특수문자제거, 숫자 제거, 문장 부호 제거등 가능)

nltk.download('punkt_tab') # 토큰화 도구 다운로드 
nltk.download('stopwords') # 불용어 목록 다운로드

# 불용어 정의
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\c\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\c\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
# 3. 전처리 함수
def preprocess_text(text):
    # 소문자 변환
    text = text.lower()
    # 특수문자 제거 (알파벳만 남김)
    text = re.sub(r'[^a-z\s]', '', text) # 소문자(a~z)와 공백(\s) 이외의 모든 문자
    # 토큰화
    tokens = word_tokenize(text)
    # 불용어 제거
    tokens = [t for t in tokens if t not in stop_words and len(t) > 1]
    return tokens

# 4. 모든 문서 전처리
processed_docs = [preprocess_text(doc) for doc in documents]

In [9]:
print(processed_docs[0]) # 전처리된 1번 텍스트 데이터 확인


['wondering', 'anyone', 'could', 'enlighten', 'car', 'saw', 'day', 'door', 'sports', 'car', 'looked', 'late', 'early', 'called', 'bricklin', 'doors', 'really', 'small', 'addition', 'front', 'bumper', 'separate', 'rest', 'body', 'know', 'anyone', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'production', 'car', 'made', 'history', 'whatever', 'info', 'funky', 'looking', 'car', 'please', 'email']


In [10]:
for i in range(20) :
    print(data.target_names[i])

alt.atheism
comp.graphics
comp.os.ms-windows.misc
comp.sys.ibm.pc.hardware
comp.sys.mac.hardware
comp.windows.x
misc.forsale
rec.autos
rec.motorcycles
rec.sport.baseball
rec.sport.hockey
sci.crypt
sci.electronics
sci.med
sci.space
soc.religion.christian
talk.politics.guns
talk.politics.mideast
talk.politics.misc
talk.religion.misc


# 2.특성 추출
- DTM
- TF-IDF Matrix
- Word2Vec
- Pre-trined Glove(Twitter-25)

## 1). Document_Term Matrix

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

# 1.CountVectorizer 객체 생성
# 공백으로 토큰 합쳐줄 것이다.
# max_features : 상위 n개 단어만 사용한다. 차원 축소다.
vectorizer = CountVectorizer(stop_words='english', max_features=5000)

# 전처리 토큰 리스트를 문자열로 변환
# "word1 word2 word3 ..." 형태로 만들어야 DTM 변환 가능
processed_docs_text_DTM = [" ".join(doc) for doc in processed_docs]

# DTM 생성
X_dtm = vectorizer.fit_transform(processed_docs_text_DTM)

print("DTM shape:", X_dtm.shape) # X_dtm.shape → (문서 수, 단어 수)
print("단어 예시:", vectorizer.get_feature_names_out()[:100]) 
# vectorizer.get_feature_names_out() → DTM에 포함된 단어 목록 확인
print(X_dtm[0])

DTM shape: (11314, 5000)
단어 예시: ['aa' 'aaa' 'ab' 'abc' 'ability' 'able' 'abort' 'abortion' 'abs' 'absence'
 'absolute' 'absolutely' 'abstract' 'absurd' 'abuse' 'abuses' 'ac'
 'academic' 'acceleration' 'accelerator' 'accept' 'acceptable'
 'acceptance' 'accepted' 'accepting' 'access' 'accident' 'accidental'
 'accidents' 'accomplish' 'accomplished' 'according' 'account' 'accounts'
 'accuracy' 'accurate' 'accused' 'achieve' 'achieved' 'acid' 'acknowledge'
 'aclu' 'acquire' 'acquired' 'act' 'acting' 'action' 'actions' 'active'
 'actively' 'activities' 'activity' 'acts' 'actual' 'actually' 'ad' 'adam'
 'adams' 'adaptec' 'adapter' 'add' 'added' 'adding' 'addition'
 'additional' 'address' 'addressed' 'addresses' 'addressing' 'adds'
 'adequate' 'adirondack' 'adjust' 'adl' 'administration' 'administrative'
 'administrator' 'admit' 'admitted' 'adobe' 'adopted' 'ads' 'adult'
 'adults' 'advance' 'advanced' 'advantage' 'advantages' 'advertising'
 'advice' 'advocate' 'aerospace' 'affairs' 'affect' 'a

## 2). TF-IDF Matrix 

In [12]:
# TF-IDF Matrix
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. 토큰 리스트를 문자열로 합치기
processed_docs_text_tfidf = [" ".join(doc) for doc in processed_docs]

# 2. TfidfVectorizer 객체 생성
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)

# 3. TF-IDF 행렬 생성 (학습 + 변환)
X_tfidf = tfidf_vectorizer.fit_transform(processed_docs_text_tfidf)

# 4. 결과 확인
print("TF-IDF shape:", X_tfidf.shape)
print("단어 예시:", tfidf_vectorizer.get_feature_names_out()[:10])

# 5. TF-IDF 행렬 확인 (희소행렬)
print(X_tfidf[0])  # 첫 번째 문서 TF-IDF 값 확인



TF-IDF shape: (11314, 5000)
단어 예시: ['aa' 'aaa' 'ab' 'abc' 'ability' 'able' 'abort' 'abortion' 'abs' 'absence']
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 27 stored elements and shape (1, 5000)>
  Coords	Values
  (0, 4921)	0.16873351038537243
  (0, 625)	0.5770428165281328
  (0, 3917)	0.15625122179382797
  (0, 1112)	0.13176485822537468
  (0, 1335)	0.18383353104247055
  (0, 4231)	0.20468916920700245
  (0, 2603)	0.16646417577540215
  (0, 2468)	0.170784119685386
  (0, 1381)	0.15588233225760645
  (0, 595)	0.1313562529018362
  (0, 1336)	0.21257256192951604
  (0, 3627)	0.11031449781535926
  (0, 4147)	0.1402247281544394
  (0, 63)	0.17317734259832754
  (0, 4011)	0.17980085800262652
  (0, 3776)	0.14996697456861016
  (0, 487)	0.15977177593639041
  (0, 2434)	0.08558426061403875
  (0, 2839)	0.16089162215486885
  (0, 1458)	0.17759906408487572
  (0, 4212)	0.20063576354399884
  (0, 4983)	0.1151511208274027
  (0, 3458)	0.20063576354399884
  (0, 2052)	0.1538666775895261
  (0, 2214)	0.1

## 3). Word2Vec

In [13]:
!pip install gensim

In [14]:
from gensim.models import Word2Vec


# Word2Vec 모델 학습
w2v_model = Word2Vec(
    sentences=processed_docs,  # 토큰 리스트
    vector_size=100,           # 임베딩 차원 (과제면 100~300)
    window=5,                  # 윈도우 사이즈
    min_count=3,               # 최소 등장 횟수
    workers=4,                 # CPU 병렬 처리
    sg=1                       # 1 = Skip-gram, 0 = CBOW
)
print("벡터 크기:", w2v_model.vector_size)
print("단어 수:", len(w2v_model.wv))

# 예시 단어 벡터 보기
print(w2v_model.wv['know'])


Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'


벡터 크기: 100
단어 수: 26858
[-0.11656905  0.41043544 -0.27389303  0.5141487   0.20520997  0.33602285
  0.47458583  0.58263767 -0.22269954 -0.1593332   0.49216047  0.03573945
  0.4538404  -0.3135477   0.17525211 -0.05746875  0.31972495  0.328339
  0.09182618 -0.5050486   0.30983326  0.09905732  0.14972566 -0.21949087
 -0.13248317  0.31774575  0.3078806   0.13189791 -0.30103016 -0.50623727
  0.60136956 -0.00767644  0.6557146   0.0778475  -0.14553173  0.26611197
 -0.22611283 -0.10846872 -0.26990446 -0.5599226   0.6691951  -0.1465858
  0.3682958  -0.1742139   0.29297087 -0.01009589 -0.1086015  -0.13453941
 -0.6825804  -0.03811453  0.29693714 -0.26676676  0.04211111  0.01508519
  0.02648553 -0.5527223   0.04759199 -0.18908085 -0.07902213 -0.6600583
 -0.39856854 -0.17602691  0.59625274 -0.18878831 -0.42617935  0.26071268
  0.02611624  0.00871262 -0.2880952  -0.01381291 -0.07645959  0.31380916
  0.03320921  0.7369341  -0.23880802  0.04197558  0.09309804 -0.17182995
 -0.2363331  -0.12555015  0.1706

In [15]:
#문서 벡터 만들기 : 문서 벡터를 통해서 카테고리를 유추하겠다.
def doc_vector(tokens): # 입력 받는 tokens는 document의 text data
    tokens = [t for t in tokens if t in w2v_model.wv] # text data의 단어가 w2v_model에 있으면 tokens에 넣는다.
    # print(tokens)
    if len(tokens) == 0:
        return np.zeros(100)
    return np.mean(w2v_model.wv[tokens], axis=0) # 여기서 token은 text data에 있는 단어 벡터 모음이다. # return 값은 문서 벡터



In [16]:
X_w2v = np.array([doc_vector(doc) for doc in processed_docs])
print(X_w2v[0]) # 첫번째 text data의 문서 벡터이다.  앞서서 100차원이라고 설정했으니 100개이다.
# 이 문서를 표현하는 벡터라고 보면 된다.
# 벡터끼리 가까우면 문장의 주제가 비슷하다고 볼 수 있다.


[-0.38090837  0.47254038  0.19713055  0.13550444  0.20539479  0.11761735
  0.36432046  0.00391385 -0.12137806  0.01118286  0.12410118 -0.34633318
  0.18765333 -0.07863998  0.11726198 -0.09726462 -0.11631639 -0.1288176
 -0.16996819 -0.35518011  0.06535943  0.14309473  0.22254595 -0.16164674
  0.09607749  0.21468958  0.00904886 -0.31106719 -0.01309814 -0.27859268
  0.27958462 -0.14930423  0.24738187  0.22641945  0.03664828  0.24197109
  0.07190105  0.11463904  0.02962998 -0.41165152  0.03325654  0.05039596
  0.06314963 -0.03747498 -0.07423417 -0.14692268  0.00191341  0.01045698
 -0.19523375  0.04333508  0.21030042  0.04911361 -0.03937061 -0.12493189
  0.16554125 -0.24166024  0.1053234   0.08561282  0.00999474 -0.23495927
 -0.24523336 -0.21143208  0.27546737 -0.09220283 -0.20348239  0.06868379
 -0.04843501  0.20218126 -0.32499114  0.17203185 -0.16800328  0.25248504
  0.11421384  0.17818326 -0.05849096 -0.16721199 -0.05849905 -0.13178584
 -0.29132378 -0.01516002 -0.11739565 -0.07039081  0.

## 4). Pre-trined Glove(Twitter-25) 

In [17]:
# Pre-trined Glove(Twitter-25) 사용

import os
import urllib.request
import zipfile
import sys


url = "https://nlp.stanford.edu/data/glove.twitter.27B.zip"
zip_path = "glove.twitter.27B.zip"
glove_dir = "glove.twitter.27B"
glove_file = os.path.join(glove_dir, "glove.twitter.27B.25d.txt")

# 1. 파일 존재 여부 확인

if os.path.exists(glove_file):
    print(f"GloVe 파일 이미 존재: {glove_file}")
else:
    # 2. 다운로드 진행 함수
    def show_progress(block_num, block_size, total_size):
        downloaded = block_num * block_size
        percent = downloaded / total_size * 100
        percent = min(100, percent)
        sys.stdout.write(f"\r다운로드 진행: {percent:.2f}%")
        sys.stdout.flush()


    # 3. 다운로드
    print("GloVe 다운로드 시작 (약 1GB)")
    urllib.request.urlretrieve(url, zip_path, reporthook=show_progress)
    print("\n다운로드 완료!")


    # 4. 압축 해제
    if not os.path.exists(glove_dir):
        print("압축 해제 중...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(glove_dir)
        print("압축 해제 완료!")


# 5. 확인
print("압축 해제된 파일 목록:")
print(os.listdir(glove_dir))


GloVe 파일 이미 존재: glove.twitter.27B\glove.twitter.27B.25d.txt
압축 해제된 파일 목록:
['glove.twitter.27B.100d.txt', 'glove.twitter.27B.200d.txt', 'glove.twitter.27B.25d.txt', 'glove.twitter.27B.50d.txt']


In [18]:
# 2. GloVe 25d 벡터 로드
glove_file = os.path.join(glove_dir, "glove.twitter.27B.25d.txt")
glove_dict = {}

print("GloVe 파일 로딩 중...")
with open(glove_file, 'r', encoding='utf8') as f:
    for line in f:
        values = line.strip().split()
        word = values[0]
        vector = np.array(values[1:], dtype='float32')
        glove_dict[word] = vector
print(f"단어 수: {len(glove_dict)}")

GloVe 파일 로딩 중...
단어 수: 1193514


In [19]:
# 3. 문서 벡터 생성 함수
def doc_vector_glove(tokens):
    vectors = [glove_dict[t] for t in tokens if t in glove_dict]
    if len(vectors) == 0:
        return np.zeros(25)
    return np.mean(vectors, axis=0)

# 4. 전체 문서 변환
X_glove = np.array([doc_vector_glove(doc) for doc in processed_docs])

print(X_glove.shape)  # (문서 수, 25)
print(X_glove)

(11314, 25)
[[-0.17667596  0.32242805  0.14661244 ...  0.08981314 -0.10036357
  -0.47160122]
 [ 0.07201795  0.37005344 -0.00142608 ...  0.08333968  0.10176188
  -0.61223906]
 [ 0.03852868  0.50571251  0.11323483 ...  0.10709827  0.08945512
  -0.38875204]
 ...
 [-0.23112494  0.19221558 -0.00541638 ...  0.64127189  0.25323161
  -0.55703974]
 [ 0.22007486  0.17631085 -0.27827433 ...  0.15954717  0.16790809
  -0.54380441]
 [-0.37664622  0.23516592  0.27163815 ...  0.46414351 -0.2559562
  -0.11672594]]


# 3. 모델별 학습 및 평가
- * 학습 데이터
- Document_Term Matrix : X_dtm (BOW 형태)
- TF-idf : X_tfidf (BOW 형태)
- Word2Vec : X_w2v (문서벡터형)
- Twitter-25 : X_glove (문서벡터형)
- 결과 데이터 : labels

In [20]:
#Logistic Regression

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import label_binarize

# full class 확장용 함수 (ROC/AUC 에러 방지)
def expand_proba_to_full_classes(y_proba, trained_classes, full_classes):
    full = np.zeros((y_proba.shape[0], len(full_classes)))
    for i, c in enumerate(trained_classes):
        idx = np.where(full_classes == c)[0][0]
        full[:, idx] = y_proba[:, i]
    return full


# 1. 학습/테스트 분리 (80% train / 20% test)
X_dtm_train, X_dtm_test, y_train, y_test = train_test_split(X_dtm, labels, test_size=0.2, random_state=42)
X_tfidf_train, X_tfidf_test, trash_val1, trash_val2 = train_test_split(X_tfidf, labels, test_size=0.2, random_state=42)
X_w2v_train, X_w2v_test, trash_val3, trash_val4 = train_test_split(X_w2v, labels, test_size=0.2, random_state=42)
X_glove_train, X_glove_test, trash_val5, trash_val6 = train_test_split(X_glove, labels, test_size=0.2, random_state=42)

# trash_val은 버리는 값이다.
# y_train하고 y_test와 동일하게 사용하기 때문에 trash_val로 표기했다.





# 2. Logistic Regression 학습/평가 함수

def train_and_evaluate_lr(X_train, X_test, y_train, y_test, name):
    print(f"=== Logistic Regression ({name}) ===")
    
    # OneVsRestClassifier로 다중 클래스 처리
    clf = OneVsRestClassifier(LogisticRegression(max_iter=5000, random_state=42))
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    # Accuracy & F1-score
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    # ROC/AUC 계산
    try:
        y_proba = clf.predict_proba(X_test)
        # full class 확장
        y_full = expand_proba_to_full_classes(y_proba, clf.classes_, np.unique(y_test))
        y_test_bin = label_binarize(y_test, classes=np.unique(y_test))
        roc = roc_auc_score(y_test_bin, y_full, multi_class='ovr')
    except:
        roc = None

    
    print(f"Accuracy: {acc:.4f}, F1-score: {f1:.4f}, ROC/AUC: {roc if roc is not None else 'N/A'}\n")
    return clf, acc, f1, roc


# 학습/평가 및 결과 저장
acc_lr = [0]*4
f1_lr  = [0]*4
roc_lr = [0]*4

clf_dtm_lr, acc_lr[0], f1_lr[0], roc_lr[0] = train_and_evaluate_lr(X_dtm_train, X_dtm_test, y_train, y_test, "Document-Term Matrix")
clf_tfidf_lr, acc_lr[1], f1_lr[1], roc_lr[1] = train_and_evaluate_lr(X_tfidf_train, X_tfidf_test, y_train, y_test, "TF-IDF")
clf_w2v_lr, acc_lr[2], f1_lr[2], roc_lr[2] = train_and_evaluate_lr(X_w2v_train, X_w2v_test, y_train, y_test, "Word2Vec Document Vectors")
clf_glove_lr, acc_lr[3], f1_lr[3], roc_lr[3] = train_and_evaluate_lr(X_glove_train, X_glove_test, y_train, y_test, "GloVe Twitter-25 Vectors")


=== Logistic Regression (Document-Term Matrix) ===
Accuracy: 0.6597, F1-score: 0.6613, ROC/AUC: 0.9399678154833249

=== Logistic Regression (TF-IDF) ===
Accuracy: 0.7004, F1-score: 0.6983, ROC/AUC: 0.9587913082321766

=== Logistic Regression (Word2Vec Document Vectors) ===
Accuracy: 0.5943, F1-score: 0.5797, ROC/AUC: 0.9366235892058066

=== Logistic Regression (GloVe Twitter-25 Vectors) ===
Accuracy: 0.4180, F1-score: 0.3976, ROC/AUC: 0.8782381195283273



In [21]:
#Linear SVM
from sklearn.svm import LinearSVC


# Linear SVM 학습/평가 함수
def train_and_evaluate_svc(X_train, X_test, y_train, y_test, name):
    print(f"=== Linear SVM ({name}) ===")
    clf = LinearSVC(max_iter=5000, random_state=42)  # max_iter 늘려서 수렴 문제 방지
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    # Accuracy & F1-score
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    # ROC/AUC 계산
    try:
        scores = clf.decision_function(X_test)

        # decision_function shape이 (n, classes) 라면 그대로
        # 만약 일부 클래스만 있으면 full class 확장 필요
        if scores.shape[1] != len(np.unique(y_test)):
            scores = expand_proba_to_full_classes(scores, clf.classes_, np.unique(y_test))

        y_test_bin = label_binarize(y_test, classes=np.unique(y_test))
        roc = roc_auc_score(y_test_bin, scores, multi_class='ovr')
    except:
        roc = None
    
    print(f"Accuracy: {acc:.4f}, F1-score: {f1:.4f}, ROC/AUC: {roc if roc is not None else 'N/A'}\n")
    return clf, acc, f1, roc

# 학습/평가 및 결과 저장
acc_svc = [0]*4
f1_svc  = [0]*4
roc_svc = [0]*4

clf_dtm_svc, acc_svc[0], f1_svc[0], roc_svc[0] = train_and_evaluate_svc(X_dtm_train, X_dtm_test, y_train, y_test, "Document-Term Matrix")
clf_tfidf_svc, acc_svc[1], f1_svc[1], roc_svc[1] = train_and_evaluate_svc(X_tfidf_train, X_tfidf_test, y_train, y_test, "TF-IDF")
clf_w2v_svc, acc_svc[2], f1_svc[2], roc_svc[2] = train_and_evaluate_svc(X_w2v_train, X_w2v_test, y_train, y_test, "Word2Vec Document Vectors")
clf_glove_svc, acc_svc[3], f1_svc[3], roc_svc[3] = train_and_evaluate_svc(X_glove_train, X_glove_test, y_train, y_test, "GloVe Twitter-25 Vectors")

=== Linear SVM (Document-Term Matrix) ===


C:\Users\c\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Accuracy: 0.5979, F1-score: 0.5985, ROC/AUC: 0.9055547729662374

=== Linear SVM (TF-IDF) ===
Accuracy: 0.6880, F1-score: 0.6882, ROC/AUC: 0.9406237674065961

=== Linear SVM (Word2Vec Document Vectors) ===
Accuracy: 0.6023, F1-score: 0.5883, ROC/AUC: 0.9244338706474254

=== Linear SVM (GloVe Twitter-25 Vectors) ===
Accuracy: 0.3981, F1-score: 0.3675, ROC/AUC: 0.8594266699358496



In [22]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

# Random Forest 학습/평가 함수
def train_and_evaluate_rf(X_train, X_test, y_train, y_test, name):
    print(f"=== Random Forest ({name}) ===")
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    # Accuracy & F1-score
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    # ROC/AUC (predict_proba 가능)
    try:
        y_proba = clf.predict_proba(X_test)
        y_full = expand_proba_to_full_classes(y_proba, clf.classes_, np.unique(y_test))
        y_test_bin = label_binarize(y_test, classes=np.unique(y_test))
        roc = roc_auc_score(y_test_bin, y_full, multi_class='ovr')
    except:
        roc = None
    
    print(f"Accuracy: {acc:.4f}, F1-score: {f1:.4f}, ROC/AUC: {roc if roc is not None else 'N/A'}\n")
    return clf, acc, f1, roc

# 학습/평가 및 결과 저장
acc_rf = [0]*4
f1_rf  = [0]*4
roc_rf = [0]*4

clf_dtm_rf, acc_rf[0], f1_rf[0], roc_rf[0] = train_and_evaluate_rf(X_dtm_train, X_dtm_test, y_train, y_test, "Document-Term Matrix")
clf_tfidf_rf, acc_rf[1], f1_rf[1], roc_rf[1] = train_and_evaluate_rf(X_tfidf_train, X_tfidf_test, y_train, y_test, "TF-IDF")
clf_w2v_rf, acc_rf[2], f1_rf[2], roc_rf[2] = train_and_evaluate_rf(X_w2v_train, X_w2v_test, y_train, y_test, "Word2Vec Document Vectors")
clf_glove_rf, acc_rf[3], f1_rf[3], roc_rf[3] = train_and_evaluate_rf(X_glove_train, X_glove_test, y_train, y_test, "GloVe Twitter-25 Vectors")


=== Random Forest (Document-Term Matrix) ===
Accuracy: 0.6200, F1-score: 0.6147, ROC/AUC: 0.9344713803905217

=== Random Forest (TF-IDF) ===
Accuracy: 0.6253, F1-score: 0.6206, ROC/AUC: 0.93761605110412

=== Random Forest (Word2Vec Document Vectors) ===
Accuracy: 0.5776, F1-score: 0.5723, ROC/AUC: 0.9369543842429928

=== Random Forest (GloVe Twitter-25 Vectors) ===
Accuracy: 0.4264, F1-score: 0.4178, ROC/AUC: 0.8922263794904282



In [24]:
# 4가지 벡터 방식 이름
vector_names = ["DTM", "TF-IDF", "Word2Vec", "GloVe-25"]

# 각 모델별 성능 저장
results = {
    "Vectorizer": vector_names,
    
    "LR_Acc": acc_lr,
    "LR_F1": f1_lr,
    "LR_ROC": roc_lr,

    "SVM_Acc": acc_svc,
    "SVM_F1": f1_svc,
    "SVM_ROC": roc_svc,
    
    "RF_Acc": acc_rf,
    "RF_F1": f1_rf,
    
    "RF_ROC": roc_rf,


}

df_results = pd.DataFrame(results)

print(df_results)

  Vectorizer    LR_Acc     LR_F1    LR_ROC   SVM_Acc    SVM_F1   SVM_ROC  \
0        DTM  0.659744  0.661320  0.939968  0.597879  0.598550  0.905555   
1     TF-IDF  0.700398  0.698314  0.958791  0.688025  0.688193  0.940624   
2   Word2Vec  0.594344  0.579709  0.936624  0.602298  0.588319  0.924434   
3   GloVe-25  0.418029  0.397597  0.878238  0.398144  0.367462  0.859427   

     RF_Acc     RF_F1    RF_ROC  
0  0.619973  0.614706  0.934471  
1  0.625276  0.620623  0.937616  
2  0.577552  0.572346  0.936954  
3  0.426425  0.417827  0.892226  


# 4.모델 성능 평가

- DTM : Logistic Regression > Random Forest > SVM
- TF-IDF : Logistic Regression > SVM > Random Forest
- Word2Vec : Random Forest > Logistic Regression > SVM
- Glove-25 : Random Forest > Logistic Regression > SVM

- 대략 TF-IDF > DTM > Word2Vec > GloVe-25 (DTM 하고 Word2Vec 비슷하다)

- 우선, 선형 모델에 비해 Random Forest는 비교적 안정적이나 선형 모델 대비 정확도가 낮다.
- Word2Vec, GloVe-25: 차원이 낮아 정보 손실, 단어 임베딩 기반은 더 큰 차원일 때 성능 기대 가능

- 가장 좋은 조합은 TF-IDF 와 Logistic Regression 혹은 Linear SVM 조합이다.
- (Accuray / F1 이 최고 성능을 보이고, ROC/AUC 도 높다.)